In [ ]:
pip install transformers

In [ ]:
pip install torch

In [ ]:
pip install pytorch

# GPU Test

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
torch.cuda.is_available()

# Data_loader

In [ ]:
import torch
import pandas as pd
import numpy as np
import transformers
import torchvision
from torchvision import transforms
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

import torch.nn.functional as F
from transformers import BertModel
import random
import time
import os
import re

# 预处理
def text_preprocessing(text):
   
    
    text = re.sub(r'(@.*?)[\s]', ' ', text)

    
    text = re.sub(r'&amp;', '&', text)

 
    text = re.sub(r'\s+', ' ', text).strip()

    return text


class FakeNewsDataset(Dataset):

    def __init__(self, df, root_dir, image_transform, tokenizer, MAX_LEN):
       
        self.csv_data = df
        self.root_dir = root_dir
        self.image_transform = image_transform
        self.tokenizer_bert = tokenizer
        self.MAX_LEN = MAX_LEN

    def __len__(self):
        return self.csv_data.shape[0]
    
    def pre_processing_BERT(self, sent):

        
        input_ids = []
        attention_mask = []
        
        encoded_sent = self.tokenizer_bert.encode_plus(
            text=text_preprocessing(sent), 
            add_special_tokens=True,        
            max_length=self.MAX_LEN,        
            padding='max_length',           
            # return_tensors='pt',          
            return_attention_mask=True,    
            truncation=True
            )
        
        input_ids = encoded_sent.get('input_ids')
        attention_mask = encoded_sent.get('attention_mask')
        
       
        input_ids = torch.tensor(input_ids)
        attention_mask = torch.tensor(attention_mask)
        
        return input_ids, attention_mask
     
        
    def __getitem__(self, idx):

        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_name = self.root_dir + self.csv_data['image_id'][idx] + '.jpg'
        image = Image.open(img_name).convert("RGB")
        image = self.image_transform(image)
        
        text = self.csv_data['post_text'][idx]
        tensor_input_id, tensor_input_mask = self.pre_processing_BERT(text)

        label = self.csv_data['label'][idx]

        if label == 'fake':
            label = '1'
        else:
            label = '0'
        label = int(label)
        
        label = torch.tensor(label)

        sample = {
                  'image_id'  :  image, 
                  'BERT_ip'   : [tensor_input_id, tensor_input_mask],
                  'label'     :  label
                  }

        return sample

# Models

In [ ]:
from importlib_metadata import re
import torch
import numpy as np
import transformers
import torchvision
from torchvision import models, transforms
import torch.nn as nn
from transformers import BertModel

device = torch.device("cuda")


class TextEncoder(nn.Module):

    def __init__(self, text_fc2_out=32, text_fc1_out=2742, dropout_p=0.4, fine_tune_module=False):

        super(TextEncoder, self).__init__()
        
        self.fine_tune_module = fine_tune_module

       
        self.bert = BertModel.from_pretrained(
                    'bert-base-uncased',
#                     output_attentions = True, 
                    return_dict=True)

        self.text_enc_fc1 = torch.nn.Linear(768, text_fc1_out)

        self.text_enc_fc2 = torch.nn.Linear(text_fc1_out, text_fc2_out)

        self.dropout = nn.Dropout(dropout_p)

        self.fine_tune()
        
    def forward(self, input_ids, attention_mask):
       

       
        out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # print(out['pooler_output'].shape)
        x = self.dropout(
            torch.nn.functional.relu(
                self.text_enc_fc1(out['pooler_output']))
        )    
        
        x = self.dropout(
            torch.nn.functional.relu(
                self.text_enc_fc2(x))
        ) 
        
        return x
    
    def fine_tune(self):
        
        for p in self.bert.parameters():
            p.requires_grad = self.fine_tune_module
            


class VisionEncoder(nn.Module):
   
    def __init__(self, img_fc1_out=2742, img_fc2_out=32, dropout_p=0.4, fine_tune_module=False):
        super(VisionEncoder, self).__init__()
        
        self.fine_tune_module = fine_tune_module
        
      
        vgg = models.vgg19(pretrained=True)
        vgg.classifier = nn.Sequential(*list(vgg.classifier.children())[:1])
        
        self.vis_encoder = vgg

        self.vis_enc_fc1 = torch.nn.Linear(4096, img_fc1_out)

        self.vis_enc_fc2 = torch.nn.Linear(img_fc1_out, img_fc2_out)

        self.dropout = nn.Dropout(dropout_p)

        self.fine_tune()
        
    def forward(self, images):
     
        x = self.vis_encoder(images)

        x = self.dropout(
            torch.nn.functional.relu(
                self.vis_enc_fc1(x))
        )

        x = self.dropout(
            torch.nn.functional.relu(
                self.vis_enc_fc2(x))
        )

        return x
    
    def fine_tune(self):
       
        for p in self.vis_encoder.parameters():
            p.requires_grad = False

        
        for c in list(self.vis_encoder.children())[5:]:
            for p in c.parameters():
                p.requires_grad = self.fine_tune_module


class Text_Concat_Vision(torch.nn.Module):

    def __init__(self,
        model_params
    ):
        super(Text_Concat_Vision, self).__init__()
        
        self.text_encoder = TextEncoder(model_params['text_fc2_out'], model_params['text_fc1_out'], model_params['dropout_p'], model_params['fine_tune_text_module'])
        self.vision_encode = VisionEncoder(model_params['img_fc1_out'], model_params['img_fc2_out'], model_params['dropout_p'], model_params['fine_tune_vis_module'])

        self.fusion = torch.nn.Linear(
            in_features=(model_params['text_fc2_out'] + model_params['img_fc2_out']), 
            out_features=model_params['fusion_output_size']
        )
        self.fc = torch.nn.Linear(
            in_features=model_params['fusion_output_size'], 
            out_features=1
        )
        self.dropout = torch.nn.Dropout(model_params['dropout_p'])


    #def forward(self, text, image, label=None):
    def forward(self, text, image, label=None):

        ## text to Bert
        text_features = self.text_encoder(text[0], text[1])
        ## image to vgg
        image_features = self.vision_encode(image)

        
        combined_features = torch.cat(
            [text_features, image_features], dim = 1
        )

        combined_features = self.dropout(combined_features)
        
        fused = self.dropout(
            torch.relu(
            self.fusion(combined_features)
            )
        )
        
        # prediction = torch.nn.functional.sigmoid(self.fc(fused))
        prediction = torch.sigmoid(self.fc(fused))

        prediction = prediction.squeeze(-1)

        # prediction = prediction.cpu().detach().numpy()
        
        # for i in range(len(prediction)):
        #     if prediction[i] > 0.5:
        #         prediction[i] = 1.0
        #     else:
        #         prediction[i] = 0.0

        # prediction = torch.tensor(prediction, dtype=torch.float32, requires_grad = False).to(device)
        prediction = prediction.float()
        
        return prediction

# Train

In [ ]:
import torch
import pandas as pd
import numpy as np
import transformers
import torchvision
from torchvision import models, transforms
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
import random
import time
import os
import re
import math


def train(model, loss_fn, optimizer, scheduler, train_dataloader, val_dataloader=None, epochs=4, evaluation=True, device='cpu', 
            param_dict_model=None, param_dict_opt=None, save_best=True, file_path='./best_model',
            writer=None
            ):
    
    best_acc_val = 0
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        

        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*100)

        
        t0_epoch, t0_batch = time.time(), time.time()

        total_loss, batch_loss, batch_counts = 0, 0, 0

        
        model.train()

        for step, batch in enumerate(train_dataloader):
            batch_counts +=1

            img_ip , text_ip, label = batch["image_id"], batch["BERT_ip"], batch['label']
            
            b_input_ids, b_attn_mask = tuple(t.to(device) for t in text_ip)
            
            imgs_ip = img_ip.to(device)
            
            b_labels = label.to(device)

            
            model.zero_grad()

            
            # logits, att_mask_img = model(text=[b_input_ids, b_attn_mask], image=imgs_ip, label=b_labels)
            logits = model(text=[b_input_ids, b_attn_mask], image=imgs_ip)

            b_labels=b_labels.to(torch.float32)
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            
            optimizer.step()
            scheduler.step()


            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                time_elapsed = time.time() - t0_batch

                
                print(f"epoch{epoch_i + 1:^7} | batch{step:^7} | loss{batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | elapsed{time_elapsed:^9.2f}")
                
                
                if writer != None:
                    writer.add_scalar('Training Loss', (batch_loss / batch_counts), epoch_i*len(train_dataloader)+step)
                
                
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

       
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*100)

        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            
            val_loss, val_accuracy = evaluate(model, loss_fn, val_dataloader, device)
            
            time_elapsed = time.time() - t0_epoch
            print(f" {epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
            
            if writer != None:
                writer.add_scalar('Validation Loss', val_loss, epoch_i+1)
                writer.add_scalar('Validation Accuracy', val_accuracy, epoch_i+1)
            
            
            if save_best: 
                
                if val_accuracy > best_acc_val:
                    best_acc_val = val_accuracy
                    torch.save({
                                'epoch': epoch_i+1,
                                'model_params': param_dict_model,
                                'opt_params': param_dict_opt,
                                'model_state_dict': model.state_dict(),
                                'opt_state_dict': optimizer.state_dict(),
                                'sch_state_dict': scheduler.state_dict()
                               }, file_path)
                    
        print("\n")
    
    print("Training complete!")
    
    
def evaluate(model, loss_fn, val_dataloader, device):
   

    model.eval()
    val_accuracy = []
    val_loss = []

    for batch in val_dataloader:
        img_ip , text_ip, label = batch["image_id"], batch["BERT_ip"], batch['label']
            
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in text_ip)

        imgs_ip = img_ip.to(device)

        b_labels = label.to(device)

    
        with torch.no_grad():
            # logits, att_mask_img = model(text=[b_input_ids, b_attn_mask], image=imgs_ip, label=b_labels)
            logits = model(text=[b_input_ids, b_attn_mask], image=imgs_ip)
            b_labels=b_labels.to(torch.float32)
            
        
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        logits[logits<0.5] = 0
        logits[logits>=0.5] = 1
        # print(logits)
        #
        # preds = torch.argmax(logits, dim=1).flatten()
        #print(preds)

        
        accuracy = (logits == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)
       

    
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    

    return val_loss, val_accuracy

# main

In [ ]:
import torch
import pandas as pd
import numpy as np
import transformers
import torchvision
from torchvision import models, transforms
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
import random
import time
import os
import re
import math

from torch.utils.tensorboard import SummaryWriter


#tesing.............................

from importlib_metadata import re
import torch
import numpy as np
import transformers
import torchvision
from torchvision import models, transforms
import torch.nn as nn
from transformers import BertModel

device = torch.device("cuda")

import torch
import pandas as pd
import numpy as np
import transformers
import torchvision
from torchvision import transforms
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

import torch.nn.functional as F
from transformers import BertModel
import random
import time
import os
import re

import torch
import pandas as pd
import numpy as np
import transformers
import torchvision
from torchvision import models, transforms
from PIL import Image
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel
from transformers import AdamW, get_linear_schedule_with_warmup
import random
import time
import os
import re
import math

#from models import *
#from data_loader import *
#from train_val import *


df_train = pd.read_csv("../input/fakenews/SpotFake_Implemetation-master/twitter/train_posts_clean.csv")
df_test = pd.read_csv("../input/fakenews/SpotFake_Implemetation-master/twitter/test_posts.csv")

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
    

image_transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(224, 224)),
        torchvision.transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


MAX_LEN = 500
root_dir = "../input/fakenews/SpotFake_Implemetation-master/twitter/"


transformed_dataset_train = FakeNewsDataset(df_train, root_dir+"images_train/", image_transform, tokenizer, MAX_LEN)

transformed_dataset_val = FakeNewsDataset(df_test, root_dir+"images_test/", image_transform, tokenizer, MAX_LEN)

train_dataloader = DataLoader(transformed_dataset_train, batch_size=8,
                        shuffle=True, num_workers=0)

val_dataloader = DataLoader(transformed_dataset_val, batch_size=8,
                        shuffle=True, num_workers=0)



loss_fn = nn.BCELoss()

def set_seed(seed_value=42):
    
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

parameter_dict_model={
    'text_fc2_out': 32, 
    'text_fc1_out': 2742, 
    'dropout_p': 0.4, 
    'fine_tune_text_module': False,
    'img_fc1_out': 2742, 
    'img_fc2_out': 32, 
    'dropout_p': 0.4, 
    'fine_tune_vis_module': False,
    'fusion_output_size': 35,
    'no_deprecation_warning':True
    }

parameter_dict_opt={'l_r': 3e-5,
                    'eps': 1e-8
                    }


EPOCHS=50

# 设置随机种子
set_seed(7)

final_model = Text_Concat_Vision(parameter_dict_model)

final_model = final_model.to(device) 


optimizer = AdamW(final_model.parameters(),
                  lr=parameter_dict_opt['l_r'],
                  eps=parameter_dict_opt['eps'])


total_steps = len(train_dataloader) * EPOCHS


scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0, # 默认值
                                            num_training_steps=total_steps)


writer = SummaryWriter('./summary')


train(model=final_model,
      loss_fn=loss_fn, optimizer=optimizer, scheduler=scheduler,
      train_dataloader=train_dataloader, val_dataloader=val_dataloader,
      epochs=15, evaluation=True,
      device=device,
      param_dict_model=parameter_dict_model, param_dict_opt=parameter_dict_opt,
      save_best=True,
      file_path='./best_model.pt', 
      writer=writer
      )